

---

# 🧠 Day 2 – Session 1 • Topic 3  
## GIL-Aware Timing & Concurrency Trade-offs

This script demonstrates how Python's **Global Interpreter Lock (GIL)** impacts performance when using threads vs processes for **CPU-bound workloads**.

---

## ✅ Cleaned Code with Inline Comments

```python
import time
import threading
import multiprocessing
import sys
import math

# A CPU-bound task: compute many square roots
def cpu_task(n):
    total = 0.0
    for i in range(1, n):
        total += math.sqrt(i)
    return total

# Adjust N based on your machine speed to make the demo fast or slow
N = 300_000  # ~300K iterations per run


# --------------------------
# 1. Single-thread baseline
# --------------------------
print("Running single-threaded baseline...")
t0 = time.time()
cpu_task(N)
baseline = time.time() - t0
print(f"Single-thread : {round(baseline, 3)} sec\n")


# --------------------------
# 2. Multi-threaded attempt
# --------------------------
# Threads won't run in parallel due to the GIL
def threaded_run(workers=4):
    threads = [threading.Thread(target=cpu_task, args=(N,)) for _ in range(workers)]
    for th in threads:
        th.start()
    for th in threads:
        th.join()

print("Running multi-threaded (GIL limits parallelism)...")
t0 = time.time()
threaded_run()
threaded_time = time.time() - t0
print(f"4 threads     : {round(threaded_time, 3)} sec (GIL)\n")


# --------------------------
# 3. Multi-processing version
# --------------------------
# Each process has its own GIL — true parallelism!
def mp_worker(_):
    cpu_task(N)

def multiproc_run(workers=4):
    with multiprocessing.Pool(workers) as pool:
        pool.map(mp_worker, range(workers))

print("Running multi-process (no shared GIL)...")
t0 = time.time()
multiproc_run()
mp_time = time.time() - t0
print(f"4 processes   : {round(mp_time, 3)} sec (no shared GIL)\n")


# --------------------------
# 4. sys.setswitchinterval demo
# --------------------------
# This controls how often Python switches between threads (~preemption interval)
orig_si = sys.getswitchinterval()
sys.setswitchinterval(0.001)  # Set to 1 millisecond (more frequent switching)

print("Running threads with more frequent context switch...")
t0 = time.time()
threaded_run()
new_threaded_time = time.time() - t0
print(f"Threads, switch=1ms: {round(new_threaded_time, 3)} sec")
sys.setswitchinterval(orig_si)  # Restore default


# --------------------------
# ASCII Diagram
# --------------------------
print(r"""
ASCII timeline (2 threads):

t=0   [Thread-A RUN ▓▓▓]  GIL held
      [Thread-B WAIT ..]

t=0.005 context switch -> GIL to Thread-B
      [Thread-A WAIT ..]
      [Thread-B RUN ▓▓▓]

CPU-bound threads simply alternate; they never run truly in parallel.
""")
```

---

# 🧾 Key Concepts Explained

## 🔁 What Is the GIL?

The **Global Interpreter Lock (GIL)** is a **mutex** in CPython that prevents multiple native threads from executing Python bytecodes at once.

### ⚠️ Implication:
Even on multi-core machines, **only one thread can execute Python code at a time**.

---

## 🧪 So Why Use Threads?

Use threads when:
- Your program is **I/O-bound**, e.g., waiting for disk/network
- You're doing **latency hiding**
- You want **simple concurrency** without process overhead

Examples:
- Downloading multiple files
- Reading/writing files asynchronously
- GUI event loops

---

## 🚀 When Does Multiprocessing Help?

Use **`multiprocessing`** when:
- You're doing **CPU-bound computation**
- You want to use **multiple cores**
- You need **true parallel execution**

Each process gets its own:
- Memory space
- GIL
- Python interpreter

> ✅ Ideal for data processing, scientific computing, ML training loops

---

## 🧮 Sample Output (Expected Behavior)

```
Running single-threaded baseline...
Single-thread : 0.289 sec

Running multi-threaded (GIL limits parallelism)...
4 threads     : 0.576 sec (GIL)

Running multi-process (no shared GIL)...
4 processes   : 0.301 sec (no shared GIL)

Running threads with more frequent context switch...
Threads, switch=1ms: 0.584 sec
```

> 📌 Even though threading runs slower than single-threaded (due to overhead), multiprocessing beats both!

---

## ⏱️ Best Practice Summary

| Strategy | For | Pros | Cons |
|---------|-----|------|------|
| Single-threaded | Simple programs | Easy, fast | No parallelism |
| Threading | I/O-bound tasks | Low overhead, easy | GIL blocks CPU parallelism |
| Multiprocessing | CPU-bound tasks | True parallelism | Higher memory use, inter-process communication |
| Native Extensions (e.g., NumPy) | CPU work | Fast, bypasses GIL | Requires external libraries |

---

## 🧠 Final Takeaway

- ❌ Don’t expect **speedup from threading** for **CPU-bound tasks**
- ✅ Use **multiprocessing** when you need **real parallelism**
- 🛠 You can tweak **`sys.setswitchinterval()`** to experiment with thread scheduling — but it **won’t fix GIL contention**
- 🚀 Consider **C extensions**, **subinterpreters (Python 3.12+)**, or **PyPy STM** for advanced parallelism

---


# 🧠 What is the GIL? (Global Interpreter Lock)

The **GIL** (Global Interpreter Lock) is a **mutex** in CPython that ensures only **one thread** executes Python bytecode at a time, even on multi-core systems.

---

## 🔒 Why Does the GIL Exist?

- To protect access to **Python objects** in memory
- Simplifies memory management and prevents race conditions
- Makes C extensions easier to write

---

## ❌ The Big Trade-off

> **No true parallelism in pure Python threads for CPU-bound tasks.**

Even with 4 threads on 4 cores:
- Only one thread runs Python code at a time
- Threads take turns using the GIL
- Adds overhead from context switching

---

## ✅ When Is It Not a Problem?

- For **I/O-bound** programs (e.g., waiting on disk/network)
- With **C extensions** (e.g., NumPy, Pandas) that release the GIL
- Using **multiprocessing** (each process has its own GIL)

---

## 🧩 Summary

| Feature | GIL Impact |
|--------|------------|
| Threading | No real parallelism for CPU work |
| Multiprocessing | Bypasses GIL (uses separate processes) |
| C Extensions | Can run in parallel if they release GIL |
| I/O Tasks | GIL doesn’t block wait time |

---

📌 **Bottom Line**:  
The GIL makes Python **safe and fast for single-threaded code**, but it **limits CPU-bound concurrency** in multithreaded programs. Use **`multiprocessing` or native extensions** when you need real parallelism.

In [3]:
import time
import threading
import multiprocessing
import sys
import math

# A CPU-bound task: compute many square roots
def cpu_task(n):
    total = 0.0
    for i in range(1, n):
        total += math.sqrt(i)
    return total

# Adjust N based on your machine speed to make the demo fast or slow
N = 300_000  # ~300K iterations per run


# --------------------------
# 1. Single-thread baseline
# --------------------------
print("Running single-threaded baseline...")
t0 = time.time()
cpu_task(N)
baseline = time.time() - t0
print(f"Single-thread : {round(baseline, 3)} sec\n")


# --------------------------
# 2. Multi-threaded attempt
# --------------------------
# Threads won't run in parallel due to the GIL
def threaded_run(workers=4):
    threads = [threading.Thread(target=cpu_task, args=(N,)) for _ in range(workers)]
    for th in threads:
        th.start()
    for th in threads:
        th.join()

print("Running multi-threaded (GIL limits parallelism)...")
t0 = time.time()
threaded_run()
threaded_time = time.time() - t0
print(f"4 threads     : {round(threaded_time, 3)} sec (GIL)\n")


# --------------------------
# 3. Multi-processing version
# --------------------------
# Each process has its own GIL — true parallelism!
def mp_worker(_):
    cpu_task(N)

def multiproc_run(workers=4):
    with multiprocessing.Pool(workers) as pool:
        pool.map(mp_worker, range(workers))

print("Running multi-process (no shared GIL)...")
t0 = time.time()
multiproc_run()
mp_time = time.time() - t0
print(f"4 processes   : {round(mp_time, 3)} sec (no shared GIL)\n")


# --------------------------
# 4. sys.setswitchinterval demo
# --------------------------
# This controls how often Python switches between threads (~preemption interval)
orig_si = sys.getswitchinterval()
sys.setswitchinterval(0.001)  # Set to 1 millisecond (more frequent switching)

print("Running threads with more frequent context switch...")
t0 = time.time()
threaded_run()
new_threaded_time = time.time() - t0
print(f"Threads, switch=1ms: {round(new_threaded_time, 3)} sec")
sys.setswitchinterval(orig_si)  # Restore default


# --------------------------
# ASCII Diagram
# --------------------------
print(r"""
ASCII timeline (2 threads):

t=0   [Thread-A RUN ▓▓▓]  GIL held
      [Thread-B WAIT ..]

t=0.005 context switch -> GIL to Thread-B
      [Thread-A WAIT ..]
      [Thread-B RUN ▓▓▓]

CPU-bound threads simply alternate; they never run truly in parallel.
""")

Running single-threaded baseline...
Single-thread : 0.025 sec

Running multi-threaded (GIL limits parallelism)...
4 threads     : 0.102 sec (GIL)

Running multi-process (no shared GIL)...
4 processes   : 0.164 sec (no shared GIL)

Running threads with more frequent context switch...
Threads, switch=1ms: 0.098 sec

ASCII timeline (2 threads):

t=0   [Thread-A RUN ▓▓▓]  GIL held
      [Thread-B WAIT ..]

t=0.005 context switch -> GIL to Thread-B
      [Thread-A WAIT ..]
      [Thread-B RUN ▓▓▓]

CPU-bound threads simply alternate; they never run truly in parallel.

